In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
#import pandas as pd
#####################################################################################
data = 'tem'
target_X= pd.read_csv(f"../data/{data}_train_input_7.csv").iloc[:,1:].values.astype(np.float32)
target_y =pd.read_csv(f"../data/{data}_train_output_7.csv").iloc[:,1:].values.astype(np.float32)


target_X_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
target_y_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)


target_X = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)
test_X= pd.read_csv(f"../data/{data}_val_input_7.csv").iloc[:,1:].values.astype(np.float32)
test_y =pd.read_csv(f"../data/{data}_val_output_7.csv").iloc[:,1:].values.astype(np.float32)

np.random.seed(2)
random_indices1 = np.random.choice(pd.read_csv("../data/M4_train.csv").iloc[:,(1):].index, size=target_X.shape[0]*20, replace=False)
X_train = pd.read_csv("../data/M4_train.csv").iloc[:,1+(24*0):].loc[random_indices1].values
y_train = pd.read_csv("../data/M4_test.csv").iloc[:,1:].loc[random_indices1].values
X_train.shape[1], y_train.shape[1]

target_X.shape,test_X.shape



###################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]

#################################################################################
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1], y_train.shape[1],1,1,256

#################################################################################
# nbeats + I모델 생성 함수
def bulid_model(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.TREND_BLOCK,
                                    NBeatsKeras.TREND_BLOCK,
                                    
                                    NBeatsKeras.TREND_BLOCK,
                                   NBeatsKeras.SEASONALITY_BLOCK,
                                   NBeatsKeras.SEASONALITY_BLOCK,
                                   NBeatsKeras.SEASONALITY_BLOCK)
                   ,nb_blocks_per_stack=1, thetas_dim=(1,2,3,24,12,6),
                   share_weights_in_stack=True, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + G모델 생성 함수    
def bulid_model_G(backcast_,forecast_,input_dim,output_dim,unit):
    model= NBeatsKeras(backcast_length=backcast_, 
                       forecast_length=forecast_,
                       input_dim=input_dim,
                       output_dim=output_dim,
                       stack_types=(NBeatsKeras.GENERIC_BLOCK,NBeatsKeras.GENERIC_BLOCK)
                   ,nb_blocks_per_stack=5, thetas_dim=(4,4),
                   share_weights_in_stack=False, hidden_layer_units=unit)
    return model 
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[select]
        y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 0, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys
#################################################################################
# nbeats + I모델 부트스트랩 샘플링 배깅

def train_bagging_models_G(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model_G(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        X_bootstrap = X_train[select]
        y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, restore_best_weights=True)
        history = model.fit(X_bootstrap, y_bootstrap, batch_size = batch_size_,
                  epochs=epochs_, verbose=0, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################
##########################################################################################
# 트랜스퍼 레이어
class PositionalEncoding(layers.Layer):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = layers.Dropout(rate=dropout)

        position = np.arange(max_len)[:, np.newaxis]
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        pe[:, 1::2] = np.cos(position * div_term)
        pe = pe[np.newaxis, ...]

        self.pe = tf.constant(pe, dtype=tf.float32)

    def call(self, x):
        x = x + self.pe[:, :tf.shape(x)[1], :]
        return self.dropout(x)
##########################################################################################
# 트랜스퍼 레이어
def create_model(fn,d_model, nlayers, nhead, dropout, iw, ow,lr,pretrained_output_reshaped,inputs):
    
    
    x = layers.Dense(d_model // 2, activation='relu')(pretrained_output_reshaped)
    x = layers.Dense(d_model, activation='relu')(x)
    
    pos_encoding = PositionalEncoding(d_model, dropout)
    x = pos_encoding(x)
    
    for _ in range(nlayers):
        attn_output = layers.MultiHeadAttention(num_heads=nhead, key_dim=d_model, dropout=dropout)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)
        ffn_output = layers.Dense(d_model, activation='relu')(x)
        ffn_output = layers.Dense(d_model)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x + ffn_output)
    
    x = layers.Dense(d_model // 2, activation='relu')(x)
    x = layers.Dense(1)(x)
    x = tf.squeeze(x, axis=-1)
    
    outputs = layers.Dense((iw + ow) // 2, activation='relu')(x)
    outputs = layers.Dense(ow)(outputs)
    
    optimizer = Adam(learning_rate=lr)
    target_model = Model(inputs=inputs, outputs=outputs)
    target_model.compile(optimizer=optimizer, loss=fn)
    
    return target_model
########################################################################################################################
def transfer_(model_num,models,trainable,
              lossf,epochs_,batch_size_,pt,lr_):
    #model_mapes_G = {}
    history_mapes_G = []
    model_pred_val = []
    model_pred_test = []
    for i in range(1, model_num+1):
        K.clear_session()
        model_name = f'model_{i}'
        m,_ = models#[model_name]
        model1= m[model_name]
        # 모든 레이어를 학습 불가능하게 설정
        for layer in model1.layers[:-1]:  # 새로운 레이어 추가된 부분은 학습 가능하도록 설정
            layer.trainable = trainable
        pretrained_layers = model1.layers[:-1]
        
        pretrained_model = Model(inputs=model1.input, outputs=pretrained_layers[-1].output)
        
        inputs = Input(shape=(X_train.shape[1], 1))
        pretrained_output = pretrained_model(inputs)
        pretrained_output_reshaped = layers.Reshape((y_train.shape[1], -1))(pretrained_output)
            
        model_instance = create_model(lossf ,d_model=84, nlayers=4,nhead=4, dropout=0.2, iw=X_train.shape[1], ow=y_train.shape[1],lr=lr_
                                      ,pretrained_output_reshaped=pretrained_output_reshaped,inputs=inputs)
        early_stop = EarlyStopping(monitor='val_loss', patience=pt, verbose=0, restore_best_weights=True)
    
        history = model_instance.fit(target_X, target_y, batch_size = batch_size_,
                      epochs=epochs_, verbose=0, 
                      callbacks=[early_stop],
                     validation_data = [target_X_val,target_y_val])
        
        # 예측
        pred_val = model_instance.predict(target_X_val)
        pred_val = pred_val.reshape(-1, y_train.shape[1])
        model_pred_val.append(pred_val)
        
        pred_test = model_instance.predict(test_X)
        pred_test = pred_test.reshape(-1, y_train.shape[1])
        model_pred_test.append(pred_test)

        history_mapes_G.append(history)
        print(f"########################################################fitted {i}")
    
    return model_pred_val,model_pred_test
#################################################################################
# 예측

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

2024-10-16 18:09:30.128688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 18:09:30.277713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-16 18:09:30.277735: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-16 18:09:30.986376: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
num=10
mape_models = train_bagging_models(num,'mape',2000,10,256,0.001)
smape_models = train_bagging_models(num,SMAPE(),2000,10,256,0.001)
mase_models = train_bagging_models(num,MASE(y_train,y_train.shape[1]),2000,10,256,0.001)
mae_models = train_bagging_models(num,'mae',2000,10,256,0.001)
mse_models = train_bagging_models(num,'mse',2000,10,256,0.001)

2024-10-16 18:09:33.604715: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-16 18:09:33.604761: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-10-16 18:09:33.605264: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'########################################################Model5
'########################################################Model6
'########################################################Model7
'########################################################Model8
'########################################################Model9
'########################################################Model0
'########################################################Model1
'########################################################Model2
'########################################################Model3
'########################################################Model4
'#######################################

In [3]:
mape_pred , mape_pred2 = transfer_(num,mape_models,True, 'mape',2000,8,10,0.0001)
mase_pred ,mase_pred2 = transfer_(num,mase_models,True, MASE(target_y,y_train.shape[1]),2000,8,10,0.0001)
smape_pred,smape_pred2 = transfer_(num,smape_models,True, SMAPE(),2000,8,10,0.0001)
mae_pred ,mae_pred2= transfer_(num,mape_models,True, 'mae',2000,8,10,0.0001)
mse_pred ,mse_pred2= transfer_(num,mape_models,True, 'mse',2000,8,10,0.0001)


12/12 [==============================] - 0s 23ms/step
########################################################fitted 1
12/12 [==============================] - 0s 23ms/step
########################################################fitted 2
12/12 [==============================] - 0s 21ms/step
########################################################fitted 3
12/12 [==============================] - 0s 23ms/step
########################################################fitted 4
12/12 [==============================] - 0s 24ms/step
########################################################fitted 5
12/12 [==============================] - 0s 17ms/step
########################################################fitted 6
12/12 [==============================] - 0s 16ms/step
########################################################fitted 7
12/12 [==============================] - 0s 17ms/step
########################################################fitted 8
12/12 [==============================] - 0s 12ms

In [4]:
concat_G = np.concatenate([mape_pred,mase_pred,smape_pred,mse_pred,mae_pred])
fin_pred_G = np.median(concat_G,axis=0)
print('all',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_pred, mase_pred,mse_pred])
fin_pred_G = np.median(concat_G,axis=0)
print('best',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.nan_to_num(np.array(mse_pred), nan=0)])
fin_pred_G = np.median(concat_G,axis=0)
print('mse',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mase_pred)])
fin_pred_G = np.median(concat_G,axis=0)
print('mase',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.nan_to_num(np.array(mae_pred), nan=0)])
fin_pred_G = np.median(concat_G,axis=0)
print('mae',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mape_pred)])
fin_pred_G = np.median(concat_G,axis=0)
print('mape',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(smape_pred)])
fin_pred_G = np.median(concat_G,axis=0)
print('mape',np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_G.flatten())).round(5))

concat_mase = np.concatenate([np.nan_to_num(np.array(mase_pred), nan=0)])
fin_pred_mase = np.median(concat_mase,axis=0)
MASE= np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_mase .flatten())).round(5)

concat_mape = np.concatenate([np.nan_to_num(np.array(mape_pred), nan=0)])
fin_pred_mape = np.median(concat_mape,axis=0)
MAPE= np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_mape .flatten())).round(5)

concat_smape = np.concatenate([np.nan_to_num(np.array(smape_pred), nan=0)])
fin_pred_smape = np.median(concat_smape,axis=0)
sMAPE= np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_smape .flatten())).round(5)

concat_mae = np.concatenate([np.nan_to_num(np.array(mae_pred), nan=0)])
fin_pred_mae = np.median(concat_mae,axis=0)
MAE= np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_mae .flatten())).round(5)

concat_mse = np.concatenate([np.nan_to_num(np.array(mse_pred), nan=0)])
fin_pred_mse = np.median(concat_mse,axis=0)
MSE= np.sqrt(mean_squared_error(target_y_val.flatten(),fin_pred_mse .flatten())).round(5)

performance = np.array([MASE, MAPE,sMAPE,MAE,MSE])
beta = 3 # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([fin_pred_mase.flatten().reshape(1,-1),
                    fin_pred_mape.flatten().reshape(1,-1),
                   fin_pred_smape.flatten().reshape(1,-1),
                   fin_pred_mae.flatten().reshape(1,-1),
                   fin_pred_mse.flatten().reshape(1,-1)],axis=0)

normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights, gd)
print('exp',np.sqrt(mean_squared_error(target_y_val.flatten(),ensemble_prediction)).round(5))

all 2.60646
best 2.62502
mse 2.63518
mase 2.63419
mae 2.65654
mape 16.98337
mape 5.49544
exp 2.61774


In [5]:
concat_G = np.concatenate([mape_pred2,mase_pred2,smape_pred2,mse_pred2,mae_pred2])
fin_pred_G = np.median(concat_G,axis=0)
print('all',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([mae_pred2, mase_pred2,mse_pred2])
fin_pred_G = np.median(concat_G,axis=0)
print('best',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))
concat_G = np.concatenate([mape_pred2, mase_pred2,smape_pred2])
fin_pred_G = np.median(concat_G,axis=0)
print('org',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))
concat_G = np.concatenate([np.array(mse_pred2)])
fin_pred_G = np.median(concat_G,axis=0)
print('mse',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mase_pred2)])
fin_pred_G = np.median(concat_G,axis=0)
print('mase',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mae_pred2)])
fin_pred_G = np.median(concat_G,axis=0)
print('mae',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(mape_pred2)])
fin_pred_G = np.median(concat_G,axis=0)
print('mape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_G = np.concatenate([np.array(smape_pred2)])
fin_pred_G = np.median(concat_G,axis=0)
print('smape',np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_G.flatten())).round(5))

concat_mase = np.concatenate([np.nan_to_num(np.array(mase_pred2), nan=0)])
fin_pred_mase = np.median(concat_mase,axis=0)

concat_mape = np.concatenate([np.nan_to_num(np.array(mape_pred2), nan=0)])
fin_pred_mape = np.median(concat_mape,axis=0)

concat_smape = np.concatenate([np.nan_to_num(np.array(smape_pred2), nan=0)])
fin_pred_smape = np.median(concat_smape,axis=0)

concat_mae = np.concatenate([np.nan_to_num(np.array(mae_pred2), nan=0)])
fin_pred_mae = np.median(concat_mae,axis=0)

concat_mse = np.concatenate([np.nan_to_num(np.array(mse_pred2), nan=0)])
fin_pred_mse = np.median(concat_mse,axis=0)

#performance = np.array([MASE, MAPE,sMAPE,MAE,MSE])
beta = 3 # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([fin_pred_mase.flatten().reshape(1,-1),
                    fin_pred_mape.flatten().reshape(1,-1),
                   fin_pred_smape.flatten().reshape(1,-1),
                   fin_pred_mae.flatten().reshape(1,-1),
                   fin_pred_mse.flatten().reshape(1,-1)],axis=0)

normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights, gd)
pd.DataFrame(ensemble_prediction.flatten()).to_csv('exp7/NBEATs_I+T.csv')
print('exp',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction)).round(5))

all 2.6688
best 2.63349
org 4.80088
mse 2.69222
mase 2.6452
mae 2.64189
mape 18.01192
smape 5.90831
exp 2.63527


In [6]:
eee = []
for i in range(1,20):
    weights = np.exp(-i* performance)
    normalized_weights = weights / np.sum(weights)
    ensemble_prediction = np.dot(normalized_weights, gd.reshape(5,-1))
    eee.append((i,np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten().round(5)))))
    #print(f'exp_beta{i}',np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction.flatten())).round(5))

eee

[(1, 2.6363857),
 (2, 2.6351442),
 (3, 2.6352684),
 (4, 2.6353705),
 (5, 2.6354718),
 (6, 2.635574),
 (7, 2.6356773),
 (8, 2.6357815),
 (9, 2.6358864),
 (10, 2.6359923),
 (11, 2.6360986),
 (12, 2.6362057),
 (13, 2.6363134),
 (14, 2.6364214),
 (15, 2.6365302),
 (16, 2.6366389),
 (17, 2.6367483),
 (18, 2.6368577),
 (19, 2.6369674)]